In [52]:
import os

In [53]:
%pwd

'd:\\Machine Learning\\VisaEligibilityPredictor-AU'

In [54]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelBuildingConfig:
      root_dir: Path
      x_train_file_path: Path
      y_train_file_path: Path
      c: int
      gamma: str
      kernel: str
      model: str

In [55]:
from src.visaPrediction.utils import *
from src.visaPrediction.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config = CONFIG_FILE_PATH,
            schema = SCHEMA_FILE_PATH,
            params = PARAMS_FILE_PATH
      ):
            self.config = load_yaml_file(config)
            self.schema = load_yaml_file(schema)
            self.params = load_yaml_file(params)

            create_directory([self.config.artifact_root])
      
      def get_model_train_config(self) -> ModelBuildingConfig:
            config = self.config.model_building
            params = self.params.SVM

            create_directory([config.root_dir])

            model_train_config = ModelBuildingConfig(
                  root_dir=config.root_dir,
                  x_train_file_path=config.x_train_file,
                  y_train_file_path=config.y_train_file,
                  c = params.C,
                  gamma= params.gamma,
                  kernel=params.kernel,
                  model=config.model,
            )

            return model_train_config

In [56]:
import os
import joblib
from pathlib import Path
from src.visaPrediction import logger
from sklearn.svm import SVC


class ModelBuilding:
      def __init__(self, config: ModelBuildingConfig):
            self.config = config
      
      def load_file(self, file:Path):
            with open(file, 'rb') as f:
                  data = joblib.load(f)
            return data

      def build_model(self) -> None:
            x_train = self.load_file(self.config.x_train_file_path)
            y_train = self.load_file(self.config.y_train_file_path)

            model = SVC(
                  C=self.config.c,
                  gamma=self.config.gamma,
                  kernel=self.config.kernel
            )

            model.fit(x_train, y_train)

            model_path = os.path.join(self.config.root_dir, self.config.model)
            with open(model_path, "wb") as f:
                  joblib.dump(model, f)
            logger.info(f"Model building successfully in: {model}")


In [57]:
try:
      config = ConfigurationManager()
      model_training_config = config.get_model_train_config()
      model_train = ModelBuilding(config=model_training_config)
      model_train.build_model()
except Exception as e:
      raise e

[2026-02-14 12:10:09,919: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2026-02-14 12:10:09,924: INFO: __init__: yaml file: schema.yaml loaded successfully]
[2026-02-14 12:10:09,933: INFO: __init__: yaml file: params.yaml loaded successfully]
[2026-02-14 12:10:09,935: INFO: __init__: File directory: artifact created successfully]
[2026-02-14 12:10:09,935: INFO: __init__: File directory: artifact/model_building created successfully]


[2026-02-14 12:10:30,734: INFO: 1742697490: Model building successfully in: SVC(C=1)]
